In [69]:
import os
from collections import Counter
# from skimage import io
# from skimage.transform import resize
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
import h5py
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D


train_dir = './input/train'
test_dir = './input/test'

In [65]:
X = []
y = []

x_width = 50
x_height = 50

img_cnt = 0
for img in os.listdir(train_dir):
    img_cnt = img_cnt + 1
    img_name = img[0:3]
    if(img_name=='cat'):
        y.append(0)
    else:
        y.append(1)    
    img = Image.open(train_dir+'/'+img)
    img = img.resize((x_width, x_height), Image.ANTIALIAS)
    X.append(np.asarray(img)/255)
    if(img_cnt%1000==0):
        print('{} files read'.format(img_cnt))   

1000 files read
2000 files read
3000 files read
4000 files read
5000 files read
6000 files read
7000 files read
8000 files read
9000 files read
10000 files read
11000 files read
12000 files read
13000 files read
14000 files read
15000 files read
16000 files read
17000 files read
18000 files read
19000 files read
20000 files read
21000 files read
22000 files read
23000 files read
24000 files read
25000 files read


In [66]:
y = np.asarray(y)
print(y.shape)
X = np.asarray(X,dtype=np.float16)
print(X.shape)

(25000,)
(25000, 50, 50, 3)


In [37]:
# del(X)

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False)
X = preprocess_input(X)
X = model.predict(X)

In [38]:
print(X.shape)

(25000, 1, 1, 512)


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

# del(X)

(20000, 50, 50, 3)
(2500, 50, 50, 3)
(2500, 50, 50, 3)
(20000,)
(2500,)
(2500,)


In [40]:
X_reshape = np.reshape(X_train,[X_train.shape[0], -1])
print(X_reshape.shape)

(20000, 512)


In [63]:
fc = Sequential()
fc.add(Dense(200, input_dim=X_reshape.shape[1], activation=None))
fc.add(Dense(1, activation='sigmoid'))
fc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

fc.fit(X_reshape, y_train, epochs=250, batch_size=256, callbacks=[],  verbose=1)

Epoch 1/250
20000/20000 [==============================] - ETA: 55s - loss: 3.9339 - acc: 0.5781 - ETA: 5s - loss: 1.8473 - acc: 0.4991  - ETA: 4s - loss: 1.6695 - acc: 0.5046 - ETA: 3s - loss: 1.4603 - acc: 0.5009 - ETA: 1s - loss: 1.2194 - acc: 0.4989 - ETA: 1s - loss: 1.1430 - acc: 0.5001 - ETA: 0s - loss: 1.0727 - acc: 0.4970 - ETA: 0s - loss: 0.9767 - acc: 0.5021

In [64]:
score = fc.evaluate(np.reshape(X_test,[X_test.shape[0], -1]), y_test, verbose=0)
print("%s: %.2f%%" % (fc.metrics_names[1], score[1]*100))

acc: 58.04%


In [79]:
conv_net = Sequential()

# convolution layer 1
conv_net.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50,50,3)))
conv_net.add(MaxPooling2D(pool_size=(3,3)))
conv_net.add(Dropout(0.5))

print(conv_net.output.shape)

# convolution layer 2
conv_net.add(Conv2D(64, (3, 3), activation='relu'))
conv_net.add(MaxPooling2D(pool_size=(3,3)))
conv_net.add(Dropout(0.5))

print(conv_net.output.shape)

# fully connected
conv_net.add(Flatten())
conv_net.add(Dense(128, activation='relu'))
conv_net.add(Dropout(0.5))
conv_net.add(Dense(1, activation='sigmoid'))

conv_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

(?, 16, 16, 32)
(?, 4, 4, 64)


In [80]:
conv_net.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
20000/20000 [==============================] - ETA: 734s - loss: 0.7375 - acc: 0.5312 - ETA: 192s - loss: 0.7196 - acc: 0.5156 - ETA: 115s - loss: 0.7069 - acc: 0.5357 - ETA: 76s - loss: 0.7144 - acc: 0.5312  - ETA: 58s - loss: 0.7186 - acc: 0.5250 - ETA: 47s - loss: 0.7123 - acc: 0.5345 - ETA: 38s - loss: 0.7129 - acc: 0.5195 - ETA: 34s - loss: 0.7106 - acc: 0.5156

In [81]:
score = conv_net.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (conv_net.metrics_names[1], score[1]*100))

acc: 82.72%
